In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import cupy as np
from NN import NeuralNetwork
from layers import DenseLayer, FlattenLayer, CnnLayer, MaxPoolingLayer
from lossFuncs import sparse_categorical_crossentropy
from activationFuncs import relu, softmax


In [2]:
X, y = load_digits(return_X_y=True)
X = X.reshape(-1, 8, 8, 1)
X = X / 255
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
model = NeuralNetwork((8, 8, 1), lr=0.01, loss=sparse_categorical_crossentropy)
model.add_layer(CnnLayer,
                kernel_size=(4, 4),
                filters=2,
                stride=1,
                padding=1,
                activation=relu,
                input_channels=1)
model.add_layer(MaxPoolingLayer, pool_size=2, stride=2)
model.add_layer(FlattenLayer)
model.add_layer(DenseLayer, output_size=10, activation=softmax)

In [4]:
model.train(X_train, y_train, epochs=35, batch_size=100)

Training: 100%|██████████| 35/35 [29:09<00:00, 49.98s/it, loss=0.356]


In [60]:
def report_metrics(y_true, y_pred):
    predicted_classes = np.argmax(y_pred, axis=1)
    
    # Initialize variables to calculate precision and recall
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0

    # Compute TP, FP, FN, TN
    for true_class, pred_class in zip(y_true, predicted_classes):
        if pred_class == true_class:
            if pred_class == 1:
                true_positives += 1
            else:
                true_negatives += 1
        else:
            if pred_class == 1:
                false_positives += 1
            else:
                false_negatives += 1

    # Calculating metrics
    accuracy = (true_positives + true_negatives) / len(y_test)
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    print(f"Accuracy: {accuracy*100:.2f}%")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

In [61]:
preds = model(X_test)
report_metrics(y_test, preds)

Accuracy: 87.78%
Precision: 0.91
Recall: 0.32


In [50]:
from PIL import Image
import os
def load_images_to_array(image_folder):
    """load images from a folder into a numpy array"""
    image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.png', '.jpg', '.jpeg'))]
    images = []
    labels = []
    
    for image_file in image_files:
        with Image.open(image_file) as img:
            img = img.convert('L')  # Convert to greyscale
            images.append(np.array(img))
            labels.append(int(image_file.split(os.sep)[-1].split(".")[0]))
    
    return np.array(images), np.array(labels)

In [54]:
image_folder = 'pseudo-mnist'
loaded_images, labels = load_images_to_array(image_folder)
loaded_images = (1-loaded_images.reshape(-1, 8, 8, 1))/255

In [62]:
pseudo_preds = model(loaded_images)
report_metrics(labels, pseudo_preds)

Accuracy: 0.83%
Precision: 0.17
Recall: 0.33
